# Main Notebook for running code

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


## Global Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, recall_score, f1_score
from Functions.implementations import *
from sklearn.metrics import f1_score

## Load Data

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleCNN().to(device)
train_loader,valid_loader, test_loader = get_data_loaders()

Train and evaluate a simple model to test if everything works.

## SGD :

In [5]:
import os

#only for google Colab :

# Set a path in your Google Drive
csv_path = "/content/drive/MyDrive/OptiML/repo/OptML-project_SGD_ResNet_Transform.csv"
save_path = "/content/drive/MyDrive/OptiML/repo/OptML-project/Results/SGD"
# For local machine, set the path to your desired location
#save_path = os.getcwd() + "/Results/SGD"
#csv_path = save_path + "/sgd_gridsearch_results.csv"


epochs = 100
eval_interval = 10

# Create the CSV with headers if it doesn't exist
if not os.path.exists(csv_path):
    columns = ["learning_rate", "momentum"] +  [f"epoch_{i}" for i in range(epochs//eval_interval, epochs + 1, eval_interval)] + ["Test"]
    pd.DataFrame(columns=columns).to_csv(csv_path, index=False)

set_seed(42) # Set a random seed for reproducibility

In [6]:
from itertools import product


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader,valid_loader, test_loader = get_data_loaders(batch_size=128)

results_grid = {}

# learning_rates = [5e-2, 1e-2, 5e-3, 1e-3]
# momentums = [0.0, 0.6, 0.9, 0.99]

learning_rates = [5e-3, 1e-3]
momentums = [0.6]

# Iterate over all combinations of lr and momentum

for lr, momentum in product(learning_rates, momentums):
    scores, model = train_and_return_evaluation_SGD(
        get_resnet18_cifar,
        lr=lr,
        momentum=momentum,
        train_loader=train_loader,
        valid_loader=valid_loader,
        test_loader=test_loader,
        device=device,
        epochs=epochs,
        eval_interval=eval_interval
    )
    results_grid[(lr, momentum)] = scores

    # Extract F1 scores only, for CSV export
    f1_scores = [f1 for (_, _, _, f1) in scores]

    row = [lr, momentum] + f1_scores

    # Append to CSV
    df_row = pd.DataFrame([row])
    df_row.to_csv(csv_path, mode='a', header=False, index=False)

    # Save the model
    torch.save(model.state_dict(), save_path + f"/ResNet_Transform_lr_{lr}_momentum_{momentum}.pth")


🔧 Training with SGD: lr=0.001, momentum=0.6
Epoch 10 | Acc=0.4748 | Recall=0.4758 | F1=0.4709
Epoch 20 | Acc=0.5968 | Recall=0.5966 | F1=0.5959
Epoch 30 | Acc=0.6342 | Recall=0.6346 | F1=0.6306
Epoch 40 | Acc=0.6986 | Recall=0.6993 | F1=0.6992
Epoch 50 | Acc=0.7170 | Recall=0.7181 | F1=0.7163
Epoch 60 | Acc=0.7514 | Recall=0.7517 | F1=0.7514
Epoch 70 | Acc=0.7572 | Recall=0.7573 | F1=0.7554
Epoch 80 | Acc=0.7854 | Recall=0.7851 | F1=0.7866
Epoch 90 | Acc=0.7874 | Recall=0.7872 | F1=0.7891
Epoch 100 | Acc=0.8046 | Recall=0.8045 | F1=0.8042

Test Set Evaluation: Acc=0.7989 | Recall=0.7989 | F1=0.7977
